# 참고
- https://pytorch.org/docs/0.4.0/nn.html
- https://pytorch.org/tutorials/
- https://ratsgo.github.io/machine%20learning/2017/10/12/terms/
- https://github.com/DSKSD/Pytorch_Fast_Campus_2018

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
from collections import OrderedDict

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# What is a PyTorch?
- Tensor와 Optimizer, Neural Net등 GPU연산에 최적화된 모듈을 이용하여 빠르게 딥러닝 모델을 구현할 수 있는 프레임워크
Facebook이 밀고 있던 lua기반의 torch를 python버전으로 포팅함.

# > Pytorch Basic
- Tensor
- autograd
- nn
- optim

## 3. nn Module
- nn.Module는 모든 neural network의 기본 class임.
- nn.Module에는 레이어와 출력을 반환하는 메서드 forward가 포함되어있음.
- 미리 포함된 forward함수에 input을 인자로 forward를 계산하고 Parameters의 backward를 계산할 수 있음. 그리고 parameter.grad를 생성함.

### 3.1 nn.Module로 Neural Network생성하기

In [16]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__() # 부모클래스 초기화
        self.linear1= nn.Linear(2,2) # nn.Linear부분도 다 nn.Module의 자식클래스
        self.linear2= nn.Linear(2,10) # nn.Linear부분도 다 nn.Module의 자식클래스

        self.sigmoid= nn.Sigmoid() 
        
    def forward(self, inputs):
        outputs= self.linear1(inputs)
        outputs= self.linear2(outputs)
        outputs= self.sigmoid(outputs)
        
        return outputs

testnn= NN()

print(testnn)
inputs=torch.FloatTensor([1,2])

NN(
  (linear1): Linear(in_features=2, out_features=2, bias=True)
  (linear2): Linear(in_features=2, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)


### 3.2 생성한 NN에 파라미터 혹은 서브 모듈에 접근
- module class의 parameters메서드는 순차적으로 parameter를 반환하는 generator를 생성

In [12]:
testnn.parameters()

<generator object Module.parameters at 0x10d2bb830>

In [13]:
for param in testnn.parameters():
    print(param)
    param.data= torch.ones(param.size())
    

Parameter containing:
tensor([[-0.0421, -0.0520,  0.0837],
        [-0.0023,  0.5047,  0.1797]])
Parameter containing:
tensor([-0.2150, -0.3487])
Parameter containing:
tensor([[-0.1185, -0.3050],
        [-0.2266,  0.0339],
        [ 0.4215,  0.3843],
        [-0.6912,  0.4384],
        [ 0.1975,  0.6707],
        [ 0.4667, -0.6443],
        [-0.6723, -0.3411],
        [ 0.6209, -0.1178],
        [ 0.3026, -0.3286],
        [ 0.6938, -0.2992]])
Parameter containing:
tensor([ 0.5303,  0.0084, -0.3725,  0.3635, -0.3753,  0.2080, -0.2042,
        -0.0775, -0.6798, -0.3371])


In [10]:
for param in testnn.parameters():
    print(param)

Parameter containing:
tensor([[ 1.,  1.,  1.],
        [ 1.,  1.,  1.]])
Parameter containing:
tensor([ 1.,  1.])


- naemd_parameters는 순차적으로 parameter name과 parameter를 반환하는 generator를 생성


In [5]:
for name, param in testnn.named_parameters():
    print('[',name,']', param)

[ linear1.weight ] Parameter containing:
tensor([[ 0.3643, -0.3121],
        [-0.1371,  0.3319]])
[ linear1.bias ] Parameter containing:
tensor([-0.6657,  0.4241])


In [14]:
for child in testnn.children():
    print(child)

Linear(in_features=3, out_features=2, bias=True)
Linear(in_features=2, out_features=10, bias=True)
Sigmoid()


### 3.3 forward
- 순차적으로 neural network를 계산함.

In [18]:
inputs= torch.randn(3,2) # nn.Module은 기본적으로 dim=0는 batch size로 봄.
print(inputs)
print("="*50)
print(testnn(inputs))

tensor([[ 0.3133, -1.1352],
        [ 0.3773, -0.2824],
        [-2.5667, -1.4303]])
tensor([[ 0.5318,  0.3234,  0.4806,  0.4540,  0.4136,  0.5646,  0.5728,
          0.5405,  0.4282,  0.3997],
        [ 0.5162,  0.3149,  0.4768,  0.4536,  0.4412,  0.5684,  0.5690,
          0.5187,  0.4586,  0.4113],
        [ 0.6406,  0.4179,  0.5490,  0.5302,  0.2468,  0.4184,  0.5849,
          0.6563,  0.2643,  0.3714]])


### 3.4 nn.Sequential()
- 여러 nn.Module을 순차적으로 처리할 수 있는 클래스를 생성.
- 여러 모듈을 간단하게 관리할 수 있음.

In [19]:
model= nn.Sequential(nn.Linear(2,2),
                    nn.Sigmoid())
print(model)
print(model(inputs))

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Sigmoid()
)
tensor([[ 0.7251,  0.6499],
        [ 0.6593,  0.6102],
        [ 0.5000,  0.9294]])


- OrderedDict로 이름과 함께 관리할 수 있음.

In [35]:
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ])) 
print(model)

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)
